In [1]:
import pandas as pd
from googletrans import Translator
from pyvi import ViTokenizer
from difflib import SequenceMatcher
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# Khởi tạo translator
translator = Translator()

#Bản tiếng anh
english_captions = pd.read_csv('samples/forrest_gump_transcript_en.csv')

#Bản tiếng việt
vn_human = pd.read_csv('samples/forrest_gump_transcript_vn.csv')

In [2]:
# Bước 1: Dịch dataset tiếng anh sang tiếng việt rồi bỏ vào column mới trong dataset tiếng anh
english_captions['vn_machine'] = english_captions['Transcript Line'].head(10).apply(
    lambda x: translator.translate(x, src='en', dest='vi').text
)

In [3]:
# Bước 2: Tokenize bản dịch ở bước 1
english_captions['vn_machine_tokenized'] = english_captions['vn_machine'].head(10).apply(ViTokenizer.tokenize)

In [4]:
# Bước 3: Tokenize phụ đề của người dịch
vn_human['vn_human_tokenized'] = vn_human['Transcript Line'].head(10).apply(ViTokenizer.tokenize)

In [5]:
# Bước 4: Tạo hàm để tính mức tương đồng Levenshtein
def similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [6]:
# Bước 5: Tạo hàm để tính điểm số BLEU
smoothing_function = SmoothingFunction()
def bleu_score(reference, candidate):
    return f"{sentence_bleu([reference.split()], candidate.split(), smoothing_function=smoothing_function.method1):.6f}"

In [7]:
# Bước 6: So sánh sự tương đồng và tính điểm số BLEU giữa 2 bản tiếng việt 
english_captions['similarity'] = english_captions.head(10).apply(
    lambda row: similarity(row['vn_machine_tokenized'], vn_human.loc[row.name, 'vn_human_tokenized']),
    axis=1
)

english_captions['bleu_score'] = english_captions.head(10).apply(
    lambda row: bleu_score(row['vn_machine_tokenized'], vn_human.loc[row.name, 'vn_human_tokenized']),
    axis=1
)

In [8]:
# Bước 7: Lưu vào CSV 
english_captions.head(10).to_csv('pyvi_comparison_results.csv', index=False)

In [9]:
# Bước 8: In CSV của kết quả ở bước 7
display(pd.read_csv('pyvi_comparison_results.csv'))

,Line Number,Transcript Line,vn_machine,vn_machine_tokenized,similarity,bleu_score
0,1,Hello. My name's Forrest... Forrest Gump.,Xin chào.Tên tôi là Forrest ... Forrest Gump.,Xin chào . Tên tôi là Forrest ... Forrest_Gump .,0.895833,0.759836
1,2,Do you want a chocolate?,Bạn có muốn một sô cô la?,Bạn có muốn một sô cô la ?,0.562500,0.067014
2,3,I could eat about a million and a half of these.,Tôi có thể ăn khoảng một triệu rưỡi trong số này.,Tôi có_thể ăn khoảng một triệu rưỡi trong số n...,0.672897,0.053444
3,4,My mama always said,Mẹ của tôi luôn nói,Mẹ của tôi luôn nói,0.666667,0.082348
4,5,life was like a box of chocolates.,Cuộc sống giống như một hộp sôcôla.,Cuộc_sống giống như một hộp sôcôla .,0.727273,0.087836
5,6,You never know what you're going to get.,Bạn không bao giờ biết những gì bạn sẽ nhận được.,Bạn không bao_giờ biết những gì bạn sẽ nhận đư...,0.800000,0.308782
6,7,Those must be comfortable shoes.,Đó phải là đôi giày thoải mái.,Đó phải là đôi giày thoải_mái .,0.811594,0.079369
7,8,I bet you could walk all day in shoes like that,Tôi cá là bạn có thể đi bộ cả ngày trong những...,Tôi cá là bạn có_thể đi bộ cả ngày trong những...,0.800000,0.150907
8,9,and not feel a thing.,Và không cảm thấy một điều.,Và không cảm_thấy một điều .,0.520000,0.052312
9,10,I wish I had shoes like that.,Tôi ước tôi có đôi giày như thế.,Tôi ước tôi có đôi giày như thế .,0.628571,0.060307
